In [1]:
import tensorflow as tf

**`tf.get_default_graph() : 获取当前默认计算图`**

In [2]:
a = tf.constant(1)

In [3]:
a.graph is tf.get_default_graph()

True

+ **`tf.get_variable() & tf.Variable()`**
+ **`tf.Graph()`**
+ **`tf.variable_scope()`**

In [4]:
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable('v', initializer=tf.zeros(shape=[1]))

g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable('v', initializer=tf.ones(shape=[1]))

In [5]:
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable('v')), "in Graph-01")

with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable('v')), "in Graph-02")

[0.] in Graph-01
[1.] in Graph-02


In [6]:
g = tf.Graph()
with g.as_default():
    zero = tf.Variable(initial_value=tf.zeros(shape=[1]), name='Var')
    one = tf.Variable(initial_value=tf.ones(shape=[1,2,3]), name='Var1')
    
with tf.Session(graph=g) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(zero))
    print(sess.run(one))

[0.]
[[[1. 1. 1.]
  [1. 1. 1.]]]


**`tf.GraphKeys`**

+ tf.GraphKeys.**GLOBAL_VARIABLES**
    + 默认的 Variable 对象集合，在分布式环境共性
    + 使用场景：持久化 Tensorflow 模型


+ tf.GraphKeys.**LOCAL_VARIABLES**
    + 每台计算机的局部变量
    + 通常用于临时变量，如：计数器
    + 使用 tf.contrib.framework.local_variable 添加到此集合


+ tf.GraphKeys.**MODEL_VARIABLES**
    + 在构建模型时，所有用于正向传播的 Variable 都将添加到这里
    + 使用 tf.contrib.framework.model_variable 添加到此集合


+ tf.GraphKeys.**TRAINABLE_VARIABLES**
    + 将有优化器训练的变量
    + tf.trainable_variables()


+ tf.GraphKeys.**SUMMARIES**
    + 日志生成相关的张量
    + Tensorflow 计算可视化


+ tf.GraphKeys.**MOVING_AVERAGE_VARIABLES**
    + 所有计算了滑动平均值的变量
    + tf.moving_average_variables


+ tf.GraphKeys.**REGULARIZATION_LOSSES**
    + 收集在图构造期间正则化损失

**`tf.GraphKeys 使用`**

**`Tensor`**

In [7]:
a = tf.constant([[1,2,3],[1,2,3],[1,2,3],[1,2,3]], name='a')
b = tf.constant([[3,4,5],[3,4,5],[3,4,5],[3,4,5]], name='b')

result = tf.add(a,b, name='add')
print(result)

Tensor("add:0", shape=(4, 3), dtype=int32)


In [8]:
result.get_shape()

TensorShape([Dimension(4), Dimension(3)])

**`tf.Session()`**

In [9]:
sess = tf.Session()
# sess.run(...)
sess.close()

In [10]:
with tf.Session() as sess:
    print(sess.run(result))

[[4 6 8]
 [4 6 8]
 [4 6 8]
 [4 6 8]]


**`tf.Tensor.eval() / sess.run(t)`**

+ Basic Usage

In [11]:
test = tf.constant([1,2,3], name='test')

In [12]:
sess = tf.Session()
with sess.as_default():
    print(sess is tf.get_default_session())
    
    print(test.eval() == sess.run(test))
    print(test.eval())
    print(sess.run(test))

True
[ True  True  True]
[1 2 3]
[1 2 3]


+ **`sess.run() 可以一步获取多个tensor中的值`**

In [13]:
a = tf.constant([1, 2, 3], name='a')
b = tf.constant([3, 4, 5], name='b')

In [14]:
add = tf.add(a, b, name='add')
mul = tf.multiply(a, b, name='mul')

In [15]:
with tf.Session() as sess:
    print(sess.run([add, mul]))
    print(add.eval(session=sess), mul.eval(session=sess))

[array([4, 6, 8]), array([ 3,  8, 15])]
[4 6 8] [ 3  8 15]


**`tf.Variable()`**

In [16]:
weights = tf.Variable(tf.random_normal([2,3], stddev=2))

weights_ = tf.Variable(weights.initialized_value() * 2.0)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(weights))
    print(sess.run(weights_))

[[ 2.543382   -1.1242118   0.19272664]
 [ 1.2435081   0.7122078   2.5087514 ]]
[[ 5.086764   -2.2484236   0.38545328]
 [ 2.4870162   1.4244156   5.017503  ]]


In [17]:
w1 = tf.Variable(tf.random_normal([2,3],stddev=1), name='w1')
w2 = tf.Variable(tf.random_normal([2,2],stddev=1), name='w2')

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(w1))
    print(sess.run(w2))
    print()
    w1 = tf.assign(w1, w2, validate_shape=False)
    print(sess.run(w1))
    print(sess.run(w2))

[[ 0.3994433  -0.50394166  0.4995834 ]
 [ 0.5666524   1.4426688  -2.163211  ]]
[[-0.01819523 -0.12233089]
 [-0.3465839   0.09228471]]

[[-0.01819523 -0.12233089]
 [-0.3465839   0.09228471]]
[[-0.01819523 -0.12233089]
 [-0.3465839   0.09228471]]


**`tf.placeholder() & feed_dict=`**

In [18]:
w1 = tf.Variable(tf.random_normal([3,2], stddev=1))
w2 = tf.Variable(tf.random_normal([1,3], stddev=1))

x = tf.placeholder(tf.float32, shape=(2,3), name="input")
a = tf.matmul(w1, x)
y = tf.matmul(w2, a)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run(y, feed_dict={
        x : [[0.7,0.5,0.5],[0.9,0.4,0.8]]
    })
    print(res)

[[1.0787733  0.60622525 0.8768816 ]]


**`cross_entropy & tf.train.XXX`**

In [19]:
w1 = tf.Variable(tf.random_normal([3,2], stddev=1))
w2 = tf.Variable(tf.random_normal([1,3], stddev=1))

x = tf.placeholder(tf.float32, shape=(2,3), name="input")
a = tf.matmul(w1, x)
y = tf.matmul(w2, a)
y_ = [[-1.5, -0.5, 1]]

cross_entropy = tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))

learning_rate = 0.001
train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(cross_entropy)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run(y, feed_dict={
        x : [[0.7,0.5,0.5],[0.9,0.4,0.8]]
    })
    print(res)
    tf.trainable_variables()

[[-3.114949  -2.2617693 -2.2011478]]


**`完整神经网络训练样例`**

In [20]:
import tensorflow as tf
from numpy.random import RandomState

batch_size=8

w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

x = tf.placeholder(tf.float32,shape=(None,2),name='x_input')
y_ = tf.placeholder(tf.float32,shape=(None,1),name ='y_input')

a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

#tf.clip_by_value()可以将计算的数值限制在一个范围内（1e-10~1.0）
#y_表示真实值，y表示预测值，定义的是交叉熵损失函数
#对于回归问题，最常用的损失函数是均方误差（MSE）mse = tf.reduce_mean(tf.square(y_-y))
cross_entropy = -tf.reduce_mean(
            y_*tf.log(tf.clip_by_value(y,1e-10,1.0)))
#多分类问题适合softmax+cross_entrpy
#cross_entropy2 = tf.nn.softmax_cross_entropy_with_logits(y,y_)
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

rdm = RandomState(1)
dataset_size=128
X = rdm.rand(dataset_size,2)
Y = [[int(x1+x2<1)] for (x1,x2) in X]

with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    
    steps = 5000
    for i in range(steps):
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size,dataset_size)
        
        sess.run(train_step,feed_dict = {x:X[start:end],y_:Y[start:end]})
        if i%1000==0:
            total_cross_entropy = sess.run(
                cross_entropy,feed_dict={x:X,y_:Y})
            print("After %d training_step(s) ,cross_entropy on all data is %g"%(i,total_cross_entropy))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
After 0 training_step(s) ,cross_entropy on all data is 0.0674925
After 1000 training_step(s) ,cross_entropy on all data is 0.0163385
After 2000 training_step(s) ,cross_entropy on all data is 0.00907547
After 3000 training_step(s) ,cross_entropy on all data is 0.00714436
After 4000 training_step(s) ,cross_entropy on all data is 0.00578471
